In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader as api
from tqdm import tqdm

### Tokenize Reviews
Tokensize each review and convert them to pre-trained word2vec embeddings

In [3]:
# Read reviews and labels
df = pd.read_csv('data/reviews_v1.csv')
X = df['text'].to_list()
y = df['food'].values

In [4]:
# # Download word2vec model
# model = api.load('word2vec-google-news-300')

In [5]:
# # Save model
# model.save('word2vec/word2vec-google-news-300.model')

In [6]:
# Load word2vec model
model = KeyedVectors.load('word2vec/word2vec-google-news-300.model')

In [1]:
## FIXME: properly tokenize words 

In [27]:
# Tokensize each review and convert to word2vec
reviews_all = []
for review in tqdm(X, desc='Processing reviews: '):
    review_embeddings = []
    for word in review:
        try:
            review_embeddings.append(model[word])
        except KeyError: # for unknown word, use vector of zeros
            review_embeddings.append(np.zeros(300))
            
    reviews_all.append(torch.tensor(review_embeddings))

Processing reviews:   0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\mattc\AppData\Local\Temp\ipykernel_11600\1497515387.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  reviews_all.append(torch.tensor(review_embeddings))
Processing reviews: 100%|██████████| 10000/10000 [02:27<00:00, 67.92it/s]


In [28]:
# Use PyTorch pad_sequence() so that all review sequences are the same length
# Output is shape (# reviews, len longest review, dimension embedding (300))
padded_reviews = pad_sequence(reviews_all, padding_value = -1, batch_first=True) 

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 119952000000 bytes.